# Understanding VAAS Scores: S_F, S_P, and S_H

VAAS produces multiple scores during inference. While many users rely on the
final hybrid score (S_H), understanding the individual components is important
for correct interpretation and responsible use.

This notebook explains:
- what each score represents,
- how the scores differ conceptually,
- and how they should be used in practice.

No visualization is required for this notebook. The focus here is on meaning.

## Installation

Install VAAS if it is not already available in the environment.

In [ ]:
!pip install vaas==0.1.7

In [ ]:
from vaas.inference.pipeline import VAASPipeline
from PIL import Image
import requests
from io import BytesIO

## Loading the Pretrained Model

We use the same pretrained VAAS checkpoint as in previous notebooks.
This ensures consistency when comparing results across notebooks.

In [ ]:
pipeline = VAASPipeline.from_pretrained(
    "OBA-Research/vaas-v1-df2023",
    device="cpu",
    alpha=0.5,
)

## Loading an Example Image

The image is loaded from the VAAS GitHub repository to keep this notebook
fully reproducible and easy to run.

In [ ]:
image_url = (
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/"
    "examples/images/COCO_DF_C110B00000_00539519.jpg"
)

response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert("RGB")

image

## Running VAAS Inference

Calling the pipeline on an image returns all three VAAS scores along with
the anomaly map.


In [ ]:
result = pipeline(image)

In [ ]:
s_f = result["S_F"]
s_p = result["S_P"]
s_h = result["S_H"]

print(f"S_F (feature-level score): {s_f:.4f}")
print(f"S_P (patch-level score):   {s_p:.4f}")
print(f"S_H (hybrid score):        {s_h:.4f}")

## S_F: Feature-Level Inconsistency

S_F measures inconsistency at the feature representation level.
It reflects how the global and intermediate features of the image
deviate from reference statistics learned during training.

Conceptually:
- S_F is sensitive to global distribution shifts,
- it captures semantic or structural irregularities,
- and it tends to respond to broad inconsistencies across the image.

S_F alone does not provide localization.
It answers the question: “Does this image look unusual in feature space?”

---

## S_P: Patch-Level Inconsistency

S_P measures inconsistency at the patch level.
It focuses on local deviations across spatial regions of the image.

Conceptually:
- S_P is sensitive to localized artifacts,
- it responds to patch-wise irregularities,
- and it aligns more closely with spatial anomaly maps.

S_P answers a different question:
“Are there specific regions in this image that behave unusually?”

---

## S_H: Hybrid Anomaly Score

S_H is the primary output score used in most VAAS workflows.
It combines S_F and S_P into a single scalar value.

The combination is controlled by the alpha parameter:
- lower alpha emphasizes feature-level consistency,
- higher alpha emphasizes patch-level consistency.

S_H provides a balanced view of global and local inconsistency
and is best suited for ranking, triage, and high-level decisions.

---

## Why Separate Scores Matter

Providing separate scores avoids collapsing distinct signals too early.

In practice:
- S_F helps detect broad distribution shifts,
- S_P helps identify localized irregularities,
- S_H enables a unified decision signal.

This separation allows users to reason about *why* an image appears
suspicious rather than relying on a single opaque number.

---

## Practical Usage Guidelines

Common usage patterns include:
- using S_H for ranking or filtering large image collections,
- inspecting S_F and S_P when results are ambiguous,
- pairing S_H with anomaly map visualization for explanation.

VAAS scores are not probabilities and should not be interpreted
as confidence levels or binary predictions.

---

## Limitations

With the current VAAS API:
- the internal scoring components are not individually configurable,
- scores are computed in a fixed forward pass,
- alpha is set at pipeline initialization time.

These design choices prioritize simplicity and reproducibility.

---

## Summary

In this notebook, we:
- examined the meaning of S_F, S_P, and S_H,
- discussed how they differ conceptually,
- and outlined practical interpretation strategies.

The next notebook will explore how the alpha parameter
affects the balance between feature-level and patch-level scoring.

check it out: [04_effect_of_alpha_on_anomaly_scoring.ipynb](https://colab.research.google.com/drive/1IlBhIOzUEqaeqJnPJ6bWfjw0nv6BBATe?usp=sharing)
